In [ ]:
import pandas as pd
import regex as re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('full_test_data.csv')
df.fillna('', inplace=True)
# make all cols str instead of phone
df = df.astype(str)

In [ ]:
# rename text column as adres
df.rename(columns={'text': 'adres'}, inplace=True)

In [ ]:
import regex
mapping = {'Ä±': 'ı', 'ÅŸ': 'ş', 'ÄŸ': 'ğ', 'Ä°': 'İ', 'ÅŸ': 'Ş', 'Äž': 'Ğ', 'Ã§': 'ç', 'Ã¼': 'ü', 'Ã¶': 'ö', 'Ã‡': 'Ç', 'Ãœ': 'Ü', 'Ã–': 'Ö', 'Ã‡': 'Ç', 'Ã¼':'i'}

def translate_chars(string):
    for special_char, turkish_char in mapping.items():
        string = re.sub(special_char, turkish_char, string)
    return string

df = df.applymap(lambda x: translate_chars(x) if isinstance(x, str) else x)
df['adres'] = df['adres'].str.lower()


for col in df:
    df[col] = df[col].str.lower()


df['adres'].fillna('', inplace=True)
#df['street2'] = df['street2'].astype('str')
#df['street2'].fillna(' ', inplace=True)
#df['adres'] = df['street2'] + ' ' + df['address']
df['adres'] = df['adres'].apply(lambda x: regex.sub(r'[^\w\s]','',str(x)))

for col in df:
    df[col].fillna('', inplace=True)


In [ ]:

def Extract(df):
    df['New_Apartman'] = df['adres'].str.extract(r'(\w+\s(aprtman|apartmanı|apartman|apartmani|aprt|apt|Apartmani|Apartmanı|APARTMAN|konut|rezidans|kooperatif|binası|Sitesi|Site|site|SİTE|SİTESİ|SITESI|SİTELERİ|home|Apartman|QPT|SPT|toki|Evleri))', expand=True)[0]
    df['New_Sokak'] = df['adres'].str.extract(r'(\w+\s(sokak|Sokak|sok|Sok|sok.|Sok.|sokak.|Sokak.|Sokak,|sokak,|Sokak,|sok,|Sok,|sok.,|Sok.,|sokak.,|Sokak.,|sokak,|Sokak,|Sk|SOK|sk.))', expand=True)[0]
    df['New_Cadde'] = df['adres'].str.extract(r'(\w+\s(Cadde|cadde|caddesi|Caddesi|cd.|caddesi.|Cad.|cad.|Bulvar|bulvar|bulvari|bulvarı))', expand=True)[0]
    df['New_Mahalle'] = df['adres'].str.extract(r'(\w+\s(mahalle(si|)|mh|mah.|mah|MAH.|MAH|MAHALLESİ|MAHALLELERİ|Mahallesi))', expand=True)[0]
    df['Sokak_new'] = df['adres'].str.extract(r'(\d+\.\sSokak)', expand=True)[0]
    df['New_Apartman'].fillna(' ', inplace=True)
    df['New_Apartman'] = df['New_Apartman'].astype(str)
    df['New_Apartman'] = df['New_Apartman'].str.split().str[-2:]
    df['New_Apartman'] = df['New_Apartman'].apply(lambda x: ' '.join(x))
    df['New_Sokak'].fillna(df['Sokak_new'], inplace=True)
    df.drop(['Sokak_new'], axis=1, inplace=True)

    try:
        df['New_Cadde_2'] = df['New_Cadde'].str.split(' ', expand=True)[1]
    except:
        df['New_Cadde_2'] = df['New_Cadde'].str.split(' ', expand=True)[0]
    pass
    try:
        df['New_Mahalle_2'] = df['New_Mahalle'].str.split(' ', expand=True)[1]
    except:
        df['New_Mahalle_2'] = df['New_Mahalle'].str.split(' ', expand=True)[0]
    pass 

    try:
        df['New_Apartman_2'] = df['New_Apartman'].str.split(' ', expand=True)[1]
    except: 
        df['New_Apartman_2'] = df['New_Apartman'].str.split(' ', expand=True)[0]   
    pass

    try:
        df['New_Sokak_2'] = df['New_Sokak'].str.split(' ', expand=True)[1]
    except:
        df['New_Sokak_2'] = df['New_Sokak'].str.split(' ', expand=True)[0]
    pass



    df['New_Cadde'] = df['New_Cadde'].str.split(' ', expand=True)[0]
    df['New_Mahalle'] = df['New_Mahalle'].str.split(' ', expand=True)[0]
    df['New_Apartman'] = df['New_Apartman'].str.split(' ', expand=True)[0]
    df['New_Sokak'] = df['New_Sokak'].str.split(' ', expand=True)[0]



    names = ['New_Sokak_2','New_Cadde_2','New_Apartman_2','New_Mahalle_2']
    y = [x for x in names if x not in df.columns]
    # exclude yf from names
    names = [x for x in names if x not in y]
    for name in names: 
        try:
            df[name].fillna(' ', inplace=True)
        except:
            pass  


    df['New_Mahalle_2'] = df['New_Mahalle_2'].str.lower()
    df['New_Mahalle_2'].loc[df['New_Mahalle_2'].str.contains('mah', na=False)] = 'Mahallesi'
    # mh
    df['New_Mahalle_2'].loc[df['New_Mahalle_2'].str.contains('mh', na=False)] = 'Mahallesi'

    df['New_Cadde_2'] = df['New_Cadde_2'].str.lower()
    df['New_Cadde_2'].loc[df['New_Cadde_2'].str.contains('cad', na=False)] = 'Caddesi'
    # bulvar
    df['New_Cadde_2'].loc[df['New_Cadde_2'].str.contains('bulvar', na=False)] = 'Bulvarı'
    # cd
    df['New_Cadde_2'].loc[df['New_Cadde_2'].str.contains('cd', na=False)] = 'Caddesi'

    df['New_Sokak_2'] = df['New_Sokak_2'].str.lower()
    df['New_Sokak_2'].loc[df['New_Sokak_2'].str.contains('sk', na=False)] = 'Sokak'
    df['New_Sokak_2'].loc[df['New_Sokak_2'].str.contains('sok', na=False)] = 'Sokak'

    df['New_Apartman_2'] = df['New_Apartman_2'].str.lower()
    df['New_Apartman_2'].loc[df['New_Apartman_2'].str.contains('ap', na=False)] = 'Apartmanı'
    df['New_Apartman_2'].loc[df['New_Apartman_2'].str.contains('si̇te', na=False)] = 'Sitesi'
    # toki
    df['New_Apartman_2'].loc[df['New_Apartman_2'].str.contains('toki', na=False)] = 'Toki'
    # evleri
    df['New_Apartman_2'].loc[df['New_Apartman_2'].str.contains('evleri', na=False)] = 'Evleri'
    # konut
    df['New_Apartman_2'].loc[df['New_Apartman_2'].str.contains('konut', na=False)] = 'Konut'
    # sitesi
    df['New_Apartman_2'].loc[df['New_Apartman_2'].str.contains('sitesi', na=False)] = 'Sitesi'
    # si
    df['New_Apartman_2'].loc[df['New_Apartman_2'].str.contains('si', na=False)] = 'Sitesi'


    #title = ['New_Sokak', 'New_Cadde', 'New_Mahalle', 'New_Apartman']
    for i in names:
        df[i] = df[i].str.title()

    df['New_Cadde'] = df['New_Cadde'].fillna('')    
    df['New_Mahalle'] = df['New_Mahalle'].fillna('')
    df['New_Sokak'] = df['New_Sokak'].fillna('')
    df['New_Apartman'] = df['New_Apartman'].fillna('')

    # if New_Sokak has a number, add a . in the end
    df['New_Sokak'] = df['New_Sokak'].str.replace(r'(\d+)', r'\1.')
    df['New_Apartman'] = df['New_Apartman']  +' '+  df['New_Apartman_2']
    df['New_Sokak'] = df['New_Sokak']  + ' '+ df['New_Sokak_2']
    df['New_Cadde'] = df['New_Cadde']  + ' '+ df['New_Cadde_2']
    df['New_Mahalle'] = df['New_Mahalle']  +' '+  df['New_Mahalle_2']  


    # If there is 2 dot, replace with 1 dot
    df['New_Sokak'] = df['New_Sokak'].str.replace(r'(\.\.)', r'.')

    # drop New_Sokak_2	New_Apartman_2	New_Cadde_2	New_Mahalle_2
    df.drop(['New_Sokak_2', 'New_Apartman_2', 'New_Cadde_2', 'New_Mahalle_2'], axis=1, inplace=True)

    # Clean _2 from names list elements
    names_main = [x.replace('_2', '') for x in names]

    for i in names_main:
        df[i] = df[i].str.title()
    
    return df

In [ ]:
df = Extract(df)